In [1]:
from transformers import GPT2LMHeadModel
gpt2_hf = GPT2LMHeadModel.from_pretrained("gpt2") #initiating the init file of gpt2

In [2]:
gpt2_sd = gpt2_hf.state_dict()
for k,v in gpt2_sd.items():
    print(k, v.shape)

transformer.wte.weight torch.Size([50257, 768])
transformer.wpe.weight torch.Size([1024, 768])
transformer.h.0.ln_1.weight torch.Size([768])
transformer.h.0.ln_1.bias torch.Size([768])
transformer.h.0.attn.c_attn.weight torch.Size([768, 2304])
transformer.h.0.attn.c_attn.bias torch.Size([2304])
transformer.h.0.attn.c_proj.weight torch.Size([768, 768])
transformer.h.0.attn.c_proj.bias torch.Size([768])
transformer.h.0.ln_2.weight torch.Size([768])
transformer.h.0.ln_2.bias torch.Size([768])
transformer.h.0.mlp.c_fc.weight torch.Size([768, 3072])
transformer.h.0.mlp.c_fc.bias torch.Size([3072])
transformer.h.0.mlp.c_proj.weight torch.Size([3072, 768])
transformer.h.0.mlp.c_proj.bias torch.Size([768])
transformer.h.1.ln_1.weight torch.Size([768])
transformer.h.1.ln_1.bias torch.Size([768])
transformer.h.1.attn.c_attn.weight torch.Size([768, 2304])
transformer.h.1.attn.c_attn.bias torch.Size([2304])
transformer.h.1.attn.c_proj.weight torch.Size([768, 768])
transformer.h.1.attn.c_proj.bias 

In [3]:
gpt2_sd['transformer.wpe.weight'].view(-1)[:10]

tensor([-0.0188, -0.1974,  0.0040,  0.0113,  0.0638, -0.1050,  0.0369, -0.1680,
        -0.0491, -0.0565])

In [1]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-10-03 11:05:01--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.1s    

2024-10-03 11:05:01 (10.1 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [4]:
with open('./input.txt', 'r') as f:
    text = f.read()

In [10]:
data = text[:1000]
print(data)

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [13]:
import tiktoken
enc = tiktoken.get_encoding('gpt2')
tokens = enc.encode(data)
print(len(tokens[:24]))

24


So as of the next step;
we want to process these token sequences and feed them into a transformer So this means we gotta re-arrage these tokens in a way that becode idx that we can feed them into the transformer, So we dont want one very long sequesce of charactors, versus we want B independent example of, T which is batch of sequences up to T which is les than the max_seq_len 🫠

In [30]:
import torch 
buf = torch.tensor(tokens[:24+1])
x = buf[:-1].view(4,6)
y = buf[1:].view(4,6)
print(f'original LONG buf tensor was:')
print(buf)
print("-------------")
print(f'Now reshaped in B,T to be X input:')
print(x)
print("-------------")
print(f'Labels B,T+1 to be Y input:')
print(y)


original LONG buf tensor was:
tensor([ 5962, 22307,    25,   198,  8421,   356,  5120,   597,  2252,    11,
         3285,   502,  2740,    13,   198,   198,  3237,    25,   198,  5248,
          461,    11,  2740,    13,   198])
-------------
Now reshaped in B,T to be X input:
tensor([[ 5962, 22307,    25,   198,  8421,   356],
        [ 5120,   597,  2252,    11,  3285,   502],
        [ 2740,    13,   198,   198,  3237,    25],
        [  198,  5248,   461,    11,  2740,    13]])
-------------
Labels B,T+1 to be Y input:
tensor([[22307,    25,   198,  8421,   356,  5120],
        [  597,  2252,    11,  3285,   502,  2740],
        [   13,   198,   198,  3237,    25,   198],
        [ 5248,   461,    11,  2740,    13,   198]])


In [9]:
#inspect weight sharing: bottom of tf: embedding AND on top tf after final linear head
print(gpt2_sd['transformer.wte.weight'].shape)
print(gpt2_sd['lm_head.weight'].shape)

torch.Size([50257, 768])
torch.Size([50257, 768])


In [11]:
print(gpt2_sd['transformer.wte.weight'].data_ptr() == gpt2_sd['lm_head.weight'].data_ptr()) #pointing to the same spot in memory

True


In [13]:
(gpt2_sd['transformer.wte.weight'] == gpt2_sd['lm_head.weight']).all() #all identical

tensor(True)

## intuition 
if 2 embedding are similar (lower vs uppercase or same toke in different language), they tent to have similar embedding in embedding space, i.e. they presumebly have close embedding into token embed space; 
likewise at the end of the tf bloack (lm.head) if two tokens are similar semantically, we expect the prbability of them to be the same at the output of the tf. 
So both positions in the tf (at very bottom (embedding) and at the very time(lm_head) have the same idea of similar tokens should have the similar embedings/weights)